In this note, I test my codes of calculating the vector spherical harmonics.

In [1]:
from scipy.special import lpmn, lpmv
import numpy as np
from numpy import sqrt, pi, sin, cos, exp
from math import factorial
import sys

In [2]:
# Use the scipy module
l_max = 5
l = np.arange(l_max+1)
m = np.arange(l_max+1)

x = 0.5
legfunc, dlegfunc = lpmn(l_max, l_max, x)

In [3]:
def legendre_func(l, m, x):
    """Calculate the Associated Legendre function at x.
    
    Please note that here 'l' and 'm' correspond to 'm' and 'n' in the scipy.special.legendre.
    """
    
    if x > 1:
        print("For x={:.f}, |x| > 1, which is not supported.".format(x))
        sys.exit()
        
    if l < m:
        print("Illegal inpput: l={:d} < m={:d}".format(l, m))
        sys.exit()
    
    if m == l:
        
        if m:
            return (2*m -1) * sqrt(1-x**2) * legendre_func(m-1, m-1, x)
        else:
            return 1
    
    elif l - m == 1:
        return (2 * m + 1) * x * legendre_func(m, m, x)
    
    else:
        return ((2*l-1)*x*legendre_func(l-1, m, x) - (l-1+m)*legendre_func(l-2, m, x)) / (l-m)

In [4]:
print("Comparison of results")
print("             Mine    |     scipy    ")
# for (li, mi) in zip(l, m):

for li in l:
    for mi in range(li+1):
        li = int(li)
        mi = int(mi)
        print("l,m={:1d},{:1d}  {:8.2f}  {:8.2f}  ".format(
                  li, mi, legendre_func(li, mi, x), legfunc[mi,li]))

Comparison of results
             Mine    |     scipy    
l,m=0,0      1.00      1.00  
l,m=1,0      0.50      0.50  
l,m=1,1      0.87     -0.87  
l,m=2,0     -0.12     -0.12  
l,m=2,1      1.30     -1.30  
l,m=2,2      2.25      2.25  
l,m=3,0     -0.44     -0.44  
l,m=3,1      0.32     -0.32  
l,m=3,2      5.62      5.62  
l,m=3,3      9.74     -9.74  
l,m=4,0     -0.29     -0.29  
l,m=4,1     -1.35      1.35  
l,m=4,2      4.22      4.22  
l,m=4,3     34.10    -34.10  
l,m=4,4     59.06     59.06  
l,m=5,0      0.09      0.09  
l,m=5,1     -1.93      1.93  
l,m=5,2     -4.92     -4.92  
l,m=5,3     42.62    -42.62  
l,m=5,4    265.78    265.78  
l,m=5,5    460.35   -460.35  


My calculation of the Legendre function seems correct :-)

In [5]:
def B_coef(l, m, x):
    """
    """
    
    if type(l) is not int:
        
        print("The type of 'l' should be int!")
        sys.exit()
        
    elif type(m) is not int:
        print("The type of 'm' should be int!")
        sys.exit()
    
    if m == 0:
        return 0

    if l == m:
        if m == 1:
            return 1
        else:
            return (2*m - 1) * m / (m - 1) * sqrt(1-x*x) * B_coef(m-1, m-1, x)
        
    elif l - m == 1:
        return (2*m + 1) * x * B_coef(m, m, x)
    
    elif l -m >= 2:
        return ((2*l-1)*x*B_coef(l-1, m, x) - (l-1+m)*B_coef(l-2, m, x)) / (l-m)
    else:
        return 0

In [6]:
l = 5
m = 2
x = 0.5

print("B:")
print("Calculated:", B_coef(l, m, x))
print("Predicted: ", m * legendre_func(l, m, x) / sqrt(1-x*x))

B:
Calculated: -11.366583424670758
Predicted:  -11.366583424670758


The calculation of $B_{l,m}$ seems good, too.

In [7]:
def A_coef(l, m, x):
    """
    """
    
    if type(l) is not int:
        
        print("The type of 'l' should be int!")
        sys.exit()
        
    elif type(m) is not int:
        print("The type of 'm' should be int!")
        sys.exit()
    
    if m == 0:
        return sqrt(1 - x*x) * B_coef(l, 1, x)
    else:
        return (-x * l * B_coef(l, m, x) + (l+m) * B_coef(l-1, m, x)) / m

In [8]:
# print("A:")
# print("Calculated:", A_coef(l, m, x))
# print("Predicted: ", dlegfunc[-1,-1] * sqrt(1-x*x))

So does $A_{l,m}$.

In [9]:
def vec_sph_harm_calc(l, m, ra, dec):
    """
    """
    
    fac1 = (2*l+1) / l / (l+1) / (4 * pi) * factorial(l-m) / factorial(l+m)
#     fac2 = (-1)**m * sqrt(fac1) * exp(complex(0, m*ra))
    fac2 = (-1)**m * sqrt(fac1) * (cos(m*ra)+sin(m*ra)*(0+1j))
    
    x = sin(dec)
    
    facA = fac2 * A_coef(l, m, x)
    facB = fac2 * B_coef(l, m, x)

    t_vec_ra = facA
    t_vec_dc = facB * (0-1j)
    
    s_vec_ra = facB * (0+1j)
    s_vec_dc = facA
    
    
    return t_vec_ra, t_vec_dc, s_vec_ra, s_vec_dc

Then I compare my results with the spherical harmonics wrapper from scipy.special.

In [10]:
from scipy.special import sph_harm

ra = pi / 3
dec = pi / 4

vec_sph_harm_calc(l, m, ra, dec)[1]

(0.13398342629807677+0.07735536724014297j)

In [11]:
fac = (0-1j) * (-1)**m / sqrt(l * (l+1)) * m / sqrt(1-sin(dec)**2)


print("T_{l,m}:", fac * sph_harm(m, l, ra, dec))

T_{l,m}: (0.1339834262980768+0.07735536724014298j)


As it shows, these two results only differ by a factor.

In [12]:
l_max = 5
l = np.arange(l_max+1)
m = np.arange(l_max+1)

x = 0.5
legfunc, dlegfunc = lpmn(l_max, l_max, x)

fac = np.sqrt(1-x*x)
Afunc = fac * dlegfunc
Bfunc = m.reshape(l_max+1,1) * legfunc / fac

In [13]:
print("             Mine    |     scipy    ")
print("          Amn   Bmn  |  Amn   Bmn")
# for (li, mi) in zip(l, m):

for li in l:
    for mi in range(li+1):
        li = int(li)
        mi = int(mi)
        print("l,m={:1d},{:1d}  "
              "{:8.2f}  {:8.2f}  |  {:8.2f}  {:8.2f} ".format(
                  li, mi,      
              A_coef(li, mi, x), B_coef(li, mi, x),
              Afunc[mi,li], Bfunc[mi,li]))

             Mine    |     scipy    
          Amn   Bmn  |  Amn   Bmn
l,m=0,0      0.00      0.00  |      0.00      0.00 
l,m=1,0      0.87      0.00  |      0.87      0.00 
l,m=1,1     -0.50      1.00  |      0.50     -1.00 
l,m=2,0      1.30      0.00  |      1.30     -0.00 
l,m=2,1      1.50      1.50  |     -1.50     -1.50 
l,m=2,2     -2.60      5.20  |     -2.60      5.20 
l,m=3,0      0.32      0.00  |      0.32     -0.00 
l,m=3,1      5.44      0.38  |     -5.44     -0.38 
l,m=3,2      3.25     12.99  |      3.25     12.99 
l,m=3,3    -16.88     33.75  |     16.88    -33.75 
l,m=4,0     -1.35      0.00  |     -1.35     -0.00 
l,m=4,1      5.00     -1.56  |     -5.00      1.56 
l,m=4,2     29.23      9.74  |     29.23      9.74 
l,m=4,3      0.00    118.12  |      0.00   -118.12 
l,m=4,4   -136.40    272.80  |   -136.40    272.80 
l,m=5,0     -1.93      0.00  |     -1.93      0.00 
l,m=5,1     -3.81     -2.23  |      3.81      2.23 
l,m=5,2     48.31    -11.37  |     48.31    -

In [14]:
fac = np.sqrt(1-x*x)

# Initialize the Amn and Bmn
A_mat = np.zeros((l_max+1, l_max+1))
B_mat = np.zeros((l_max+1, l_max+1))
B_mat[1, 1] = 1
B_mat[1, 0] = 0

# Generate the sequence of Bmn
for l in range(2, l_max+1):
    for m in range(l+1)[::-1]:
        if m:
            if l == m:
                B_mat[l, m] = fac * (2*m-1) * m / (m-1) * B_mat[m-1, m-1]
            elif l == m+1:
                B_mat[l, m] = (2*m+1) * x * B_mat[m, m]
            else:
                B_mat[l, m] = ((2*l-1)*x*B_mat[l-1, m] - (l-1+m)*B_mat[l-2, m])
                B_mat[l, m] = B_mat[l, m] / (l-m)
        else:
            B_mat[l, m] = 0

# Calculate Amn
for l in range(1, l_max+1):
    for m in range(l+1):
        if m:
            A_mat[l, m] = (-x*l*B_mat[l, m]+(l+m)*B_mat[l-1, m]) / m
        else:
            A_mat[l, m] = fac * B_mat[l, 1]

In [15]:
print("             Mine    |     New    ")
print("          Amn   Bmn  |  Amn   Bmn")
# for (li, mi) in zip(l, m):

for li in range(l_max+1):
    for mi in range(li+1):
        li = int(li)
        mi = int(mi)
        print("l,m={:1d},{:1d}  "
              "{:8.2f}  {:8.2f}  |  "
              "{:8.2f}  {:8.2f} ".format(
                  li, mi, 
                  A_coef(li, mi, x), B_coef(li, mi, x),
                  A_mat[li,mi], B_mat[li,mi]))

             Mine    |     New    
          Amn   Bmn  |  Amn   Bmn
l,m=0,0      0.00      0.00  |      0.00      0.00 
l,m=1,0      0.87      0.00  |      0.87      0.00 
l,m=1,1     -0.50      1.00  |     -0.50      1.00 
l,m=2,0      1.30      0.00  |      1.30      0.00 
l,m=2,1      1.50      1.50  |      1.50      1.50 
l,m=2,2     -2.60      5.20  |     -2.60      5.20 
l,m=3,0      0.32      0.00  |      0.32      0.00 
l,m=3,1      5.44      0.38  |      5.44      0.38 
l,m=3,2      3.25     12.99  |      3.25     12.99 
l,m=3,3    -16.88     33.75  |    -16.88     33.75 
l,m=4,0     -1.35      0.00  |     -1.35      0.00 
l,m=4,1      5.00     -1.56  |      5.00     -1.56 
l,m=4,2     29.23      9.74  |     29.23      9.74 
l,m=4,3      0.00    118.12  |      0.00    118.12 
l,m=4,4   -136.40    272.80  |   -136.40    272.80 
l,m=5,0     -1.93      0.00  |     -1.93      0.00 
l,m=5,1     -3.81     -2.23  |     -3.81     -2.23 
l,m=5,2     48.31    -11.37  |     48.31    -11

In [16]:
from vec_sph_harm import real_vec_sph_harm_proj

l_max = 2
real_vec_sph_harm_proj(l_max, ra, dec)

(array([ 4.88602512e-01,  1.72747075e-01,  7.72548404e-01, -4.66873303e-17,
         1.57695783e-01]),
 array([ 0.        , -0.42314219,  0.        , -0.3862742 ,  0.3862742 ]),
 array([-0.00000000e+00, -2.99206710e-01, -0.00000000e+00,  8.08648282e-17,
         2.73137108e-01]),
 array([-0.        , -0.24430126, -0.        , -0.22301551, -0.22301551]))

In [19]:
from vec_sph_harm_201124 import real_vec_sph_harm_proj

for l in range(1, l_max+1):
    for m in range(l+1):
        print(l, m, real_vec_sph_harm_proj(l, m, ra, dec))

1 0 (0.24430125595146, 0.0)
1 1 (0.17274707473566778, -0.4231421876608172, -0.2992067103010745, -0.24430125595146002)
2 0 (0.38627420202318963, 0.0)
2 1 (-4.668733033545537e-17, -0.3862742020231896, 8.08648282107604e-17, -0.22301551451909643)
2 2 (0.15769578262625994, 0.3862742020231897, 0.2731371076480198, -0.22301551451909635)
